<a href="https://colab.research.google.com/github/jtao/genai/blob/main/1.%20LLM-Components.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Module 1: LLM Components

**Introduction**

Welcome to this comprehensive LLM Orchestration Teaching Kit. This notebook is designed to provide an in-depth understanding of Large Language Model (LLM) orchestration techniques. By the end of this session, you'll have hands-on experience with:

1. Advanced prompt engineering and templating
2. The inner workings of tokenizers and transformers
3. Few-shot and zero-shot learning techniques
4. Complex chain implementations using LangChain
5. Evaluation and fine-tuning of LLM outputs

This notebook is designed to take over an hour to complete, providing a deep dive into each topic with practical exercises and advanced concepts.




# Setup

First, let's install the necessary **libraries**, **import** them, and **load in our LLM**



In [ ]:
!pip install -qqq transformers langchain openai evaluate datasets tiktoken langchain-huggingface bitsandbytes accelerate huggingface_hub

In [ ]:
from huggingface_hub import login
login()

In [ ]:
# Now, let's import the required libraries:

import torch
import bitsandbytes
from transformers import AutoTokenizer, AutoModelForCausalLM, GPT2LMHeadModel, GPT2Tokenizer,TextStreamer ,pipeline
from langchain import PromptTemplate, LLMChain
from langchain_huggingface.llms import HuggingFacePipeline
from langchain.chains import SimpleSequentialChain, SequentialChain
from langchain.prompts import FewShotPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
import os
import numpy as np
from datasets import load_dataset
import evaluate
import tiktoken

In [ ]:
model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"
#model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto" , load_in_4bit=True)
streamer = TextStreamer(tokenizer)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create a text-generation pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    max_length=1024,
    truncation=True,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.15,
    streamer=streamer,
)

# Create a LangChain wrapper for the Hugging Face pipeline
llm = HuggingFacePipeline(pipeline=pipe)

# 1. Advanced Prompt Engineering and Templating

Prompt engineering is a crucial skill in working with LLMs. Let's explore various techniques to create effective prompts.

## 1.1 Simple vs. Structured Prompts

In [ ]:
# Simple prompt
simple_prompt = "Write a short story about a robot."

# Structured prompt
structured_prompt = """
Task: Write a short story
Topic: A robot
Genre: Science fiction
Length: Approximately 100 words
Key elements to include:
- The robot's purpose
- A moral dilemma the robot faces
- Resolution of the dilemma
"""

# Compare outputs
print("Simple Prompt Output:")
print(llm(simple_prompt))
print("\nStructured Prompt Output:")
print(llm(structured_prompt))

## 1.2 Advanced Templating with Dynamic Instructions

In [ ]:
from langchain import PromptTemplate

advanced_template = PromptTemplate(
    input_variables=["topic", "genre", "length", "key_elements"],
    template="""
Task: Write a short story
Topic: {topic}
Genre: {genre}
Length: Approximately {length} words
Key elements to include:
{key_elements}
"""
)

# Example usage
story_prompt = advanced_template.format(
    topic="A time-traveling historian",
    genre="Historical fiction",
    length="150",
    key_elements="- A significant historical event\n- An ethical decision about changing the past\n- Consequences of the historian's actions"
)

print(story_prompt)
print(llm(story_prompt))

## 1.3 Few-Shot Prompt Templates

In [ ]:
from langchain import PromptTemplate, FewShotPromptTemplate

# Define our list of few-shot examples
examples = [
    {"word": "happy", "antonym": "sad"},
    {"word": "tall", "antonym": "short"},
    {"word": "rich", "antonym": "poor"},
]

# Create our example template
example_template = """
Word: {word}
Antonym: {antonym}
"""

# Create a prompt example from the above template
example_prompt = PromptTemplate(
    input_variables=["word", "antonym"],
    template=example_template,
)

# Finally, create the few-shot prompt template
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="Give the antonym for each word:",
    suffix="Word: {input}\nAntonym:",
    input_variables=["input"],
    example_separator="\n\n",
)

# Test it out!
print(few_shot_prompt.format(input="big"))
print(llm(few_shot_prompt.format(input="big")))



---





# 2. Tokenizers and Transformers: A Deep Dive

## 2.1 Understanding Tokenization

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")

text = "Let's tokenize this sentence and understand how it works!"

# Tokenize the text
tokens = tokenizer.tokenize(text)
token_ids = tokenizer.encode(text)

print("Tokens:", tokens)
print("Token IDs:", token_ids)

# Decode back to text
decoded_text = tokenizer.decode(token_ids)
print("Decoded text:", decoded_text)

## 2.2 Generating Text with Transformers

In [ ]:
input_text = "In a world where AI"
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

# Generate multiple sequences
output_sequences = model.generate(
    input_ids,
    max_length=50,
    num_return_sequences=3,
    no_repeat_ngram_size=2,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.7
)

for i, seq in enumerate(output_sequences):
    print(f"Generated sequence {i + 1}:")
    print(tokenizer.decode(seq, skip_special_tokens=True))
    print()



---





# 3. Advanced Few-Shot and Zero-Shot Learning

## 3.1 Zero-Shot Classification

In [ ]:
from transformers import pipeline

classifier = pipeline("zero-shot-classification")

sequence = "I loved the new Batman movie!"
candidate_labels = ["positive", "negative", "neutral"]

result = classifier(sequence, candidate_labels)
print(result)

## 3.2 Few-Shot Learning with Dynamic Example Selection

In [ ]:
from langchain.prompts.example_selector import LengthBasedExampleSelector

# Prepare a larger set of examples
examples = [
    {"input": "I love this movie!", "output": "Positive"},
    {"input": "This film is terrible.", "output": "Negative"},
    {"input": "The acting was great but the plot was confusing.", "output": "Mixed"},
    {"input": "I've seen better.", "output": "Negative"},
    {"input": "A masterpiece of cinema!", "output": "Positive"},
    {"input": "It was okay, nothing special.", "output": "Neutral"},
]

# Create our example template
example_template = """
Input: {input}
Output: {output}
"""

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template=example_template,
)

# Create a LengthBasedExampleSelector
example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=50  # This sets the max length of examples to use
)

# Create the few-shot prompt template
dynamic_few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Classify the sentiment in these movie reviews:",
    suffix="Input: {input}\nOutput:",
    input_variables=["input"],
    example_separator="\n\n",
)

# Test it out with different inputs
print(dynamic_few_shot_prompt.format(input="This movie changed my life!"))
print(llm(dynamic_few_shot_prompt.format(input="This movie changed my life!")))

print(dynamic_few_shot_prompt.format(input="I fell asleep halfway through."))
print(llm(dynamic_few_shot_prompt.format(input="I fell asleep halfway through.")))



---






# 4. Advanced Chain Implementations with LangChain

## 4.1 Multi-Stage Processing Chain

In [ ]:
from langchain.chains import SimpleSequentialChain, LLMChain

# First chain: Generate a news headline
headline_prompt = PromptTemplate(
    input_variables=["topic"],
    template="Generate a catchy news headline about {topic}."
)
headline_chain = LLMChain(llm=llm, prompt=headline_prompt)

# Second chain: Generate a short news article
article_prompt = PromptTemplate(
    input_variables=["headline"],
    template="Write a short news article based on this headline: {headline}"
)
article_chain = LLMChain(llm=llm, prompt=article_prompt)

# Third chain: Generate three hashtags for social media
hashtag_prompt = PromptTemplate(
    input_variables=["article"],
    template="Generate three relevant hashtags for this news article: {article}"
)
hashtag_chain = LLMChain(llm=llm, prompt=hashtag_prompt)

# Combine the chains
news_chain = SimpleSequentialChain(chains=[headline_chain, article_chain, hashtag_chain], verbose=True)

# Run the chain
print(news_chain.run("artificial intelligence"))

## 4.2 Branching Chain with Conditional Logic


In [ ]:
from langchain.chains import LLMChain, SequentialChain

# Chain 1: Determine the type of task
task_prompt = PromptTemplate(
    input_variables=["objective"],
    template="Determine if the following objective requires 'analysis' or 'creation': {objective}"
)
task_chain = LLMChain(llm=llm, prompt=task_prompt)

# Chain 2A: Analysis task
analysis_prompt = PromptTemplate(
    input_variables=["objective"],
    template="Provide a detailed analysis of the following: {objective}"
)
analysis_chain = LLMChain(llm=llm, prompt=analysis_prompt)

# Chain 2B: Creation task
creation_prompt = PromptTemplate(
    input_variables=["objective"],
    template="Create a detailed plan or outline for the following: {objective}"
)
creation_chain = LLMChain(llm=llm, prompt=creation_prompt)

# Chain 3: Summarize the output
summary_prompt = PromptTemplate(
    input_variables=["result"],
    template="Summarize the key points from this output: {result}"
)
summary_chain = LLMChain(llm=llm, prompt=summary_prompt)

# Combining chains with conditional logic
def branching_chain(objective):
    task_type = task_chain.run(objective).strip().lower()
    if 'analysis' in task_type:
        result = analysis_chain.run(objective)
    else:
        result = creation_chain.run(objective)
    summary = summary_chain.run(result)
    return f"Task Type: {task_type}\n\nResult: {result}\n\nSummary: {summary}"

# Test the branching chain
print(branching_chain("Evaluate the impact of social media on modern politics"))
print("\n" + "="*50 + "\n")
print(branching_chain("Design a marketing campaign for a new electric car"))



---

